# Impactos cósmicos

Sin utilizar el aprendizaje automático, debe tratar de caracterizar las detecciones de partículas entrantes como rayos cósmicos (que se originan fuera del sistema solar, a menudo de una supernova distante) o como eyección estelar (partículas expulsadas por el sol).

Para esta prueba, solo tiene una partícula para clasificar, que impactó con una energía de 1200 MeV

Para ayudarte, tienes un conjunto de datos que contiene eyecta estelar y rayos cósmicos, ubicado en `cosmic_data.txt`. Contiene dos columnas, la primera, la energía del impacto en MeV, y la segunda, la probabilidad de que haya sido por eyección solar.

**MeV**: Mega-electronvoltio. Un MeV es un millón de veces un electronvoltio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

data = np.loadtxt("cosmic_data.txt")
test = 1200
print(data.shape)
print(data[:2, :])

In [ ]:
stellar, cosmic = data[:, 0][data[:, 1] > 0.5], data[:, 0][data[:, 1] < 0.5]

_, bins, _ = plt.hist(data[:, 0], alpha=0.2, label="Todo", bins=50)
plt.hist(stellar, bins=bins, histtype="step", label="Estelar")
plt.hist(cosmic, bins=bins, histtype="step", label="Cósmico")
plt.legend();

Esto no se ve muy bien, ¿verdad? Parece que hay dos poblaciones, sin embargo, hay contaminantes. Descartemos algunos de los datos más inciertos.

In [ ]:
stellar, cosmic = data[:, 0][data[:, 1] > 0.9], data[:, 0][data[:, 1] < 0.1]
n_stellar, n_cosmic = stellar.shape[0], cosmic.shape[0]

_, bins, _ = plt.hist(data[:, 0], alpha=0.2, label="Todo", bins=50)
plt.hist(stellar, bins=bins, histtype="step", label="Estelar")
plt.hist(cosmic, bins=bins, histtype="step", label="Cosmico")
plt.legend();

Muy bien, parece que hemos eliminado mucha contaminación. Así que ahora intentemos cuantificar cada distribución. En la mayoría de los casos, comience de manera simple y luego agregue complejidad solo cuando sea necesario.

In [ ]:
# Test Normales
params_s = st.norm.fit(stellar)
params_c = st.norm.fit(cosmic)

# visualizar
xs = np.linspace(0, 1500, 200)
p_s = st.norm.pdf(xs, *params_s)
p_c = st.norm.pdf(xs, *params_c)
plt.plot(xs, p_s, label="Modelo Estelar")
plt.plot(xs, p_c, label="Cosmic Model")
plt.hist(stellar, bins=bins, histtype="step", density=True, label="Datos Estelares")
plt.hist(cosmic, bins=bins, histtype="step", density=True, label="Datos Cosmicos")
plt.legend()
plt.xlabel("Energía (MeV)");

In [ ]:
# Test normales
params_s = st.lognorm.fit(stellar, loc=400, scale=100)
cosmic2 = cosmic[cosmic > 700]
params_c = st.lognorm.fit(cosmic2, loc=1300, scale=100)

# visualizar
xs = np.linspace(0, 1500, 200)
p_s = st.lognorm.pdf(xs, *params_s)
p_c = st.lognorm.pdf(xs, *params_c)
plt.plot(xs, p_s, label="Modelo Estelar")
plt.plot(xs, p_c, label="Datos Cosmicos")
plt.hist(stellar, bins=bins, histtype="step", density=True, label="Modelo Estelar")
plt.hist(cosmic, bins=bins, histtype="step", density=True, label="Datos Cosmicos")
plt.axvline(test, ls=":", label="Test particular")
plt.legend();

Ahora, si queremos la probabilidad de que la detección sea de un tipo de partícula u otro, no podemos simplemente tomar una proporción entre los dos, porque eso supondría que obtenemos la misma cantidad de partículas de rayos cósmicos y estelares, que no es verdadero. Parece cierto en el gráfico anterior, porque estamos trazando probabilidades, que están normalizadas a la unidad de área.

Tenemos que tener en cuenta las diferentes tasas, así:

In [ ]:
prob_cosmic = st.lognorm.pdf(test, *params_c) * n_cosmic
prob_stellar = st.lognorm.pdf(test, *params_s) * n_stellar
total_prob = prob_cosmic + prob_stellar

final = prob_cosmic / total_prob
print(f"La probabilidad final de que sea un rayo cósmico es {100 * final:.1f}%")

¡Toma esa Machine Learning! ¡No te necesitamos!